>>>>> # 팀점수를 예측하는 머신러닝 구현

In [70]:
# 필요 Module 불러오기
# sql 사용 
import pymysql
# 데이터 정제용
import pandas as pd
import numpy as np
# 머신러닝용
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from scipy import stats # 확률분포 그래프
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor # GradientBoostingRegressor
# 경고메세지무시
import warnings
warnings.filterwarnings('ignore')

### DataBase에서 데이터 불러오기

In [71]:
# mysql Connection - 요한형 DB 접속
conn = pymysql.connect(host = '192.168.101.106',user = 'root',password = '1111',db = 'asiancup',charset = 'utf8')

#Connection으로 부터 cursor생성 - 테이터 베이스 빈줄에 커서가 깜빡 거리고 있는 것이다.
curs = conn.cursor()
 
# SQL 문장
sql = 'select * from nations2011' # 2011년 매치데이터 불러오기
curs.execute(sql) # 실행문 - 결과는 데이터가 몇개 있는지 알려준다.

# 2011년 아시안컵 매치 데이터
rows = curs.fetchall()
print(rows)
conn.close() # 연결해제

((1, "['16-Jan-11', '19:15']", '카타르', 6, 23, 43, 4, 24, 92, 11295), (2, "['12-Jan-11', '19:15']", '카타르', 4, 16, 51, 1, 21, 92, 11295), (3, "['21-Jan-11', '16:25']", '카타르', 7, 18, 41, 1, 17, 92, 11295), (4, "['08-Jan-11', '16:15']", '쿠웨이트', 2, 17, 51, 1, 23, 96, 10350), (5, "['12-Jan-11', '16:15']", '쿠웨이트', 3, 14, 48, 1, 21, 96, 10350), (6, "['16-Jan-11', '19:15']", '쿠웨이트', 1, 16, 57, 4, 24, 96, 10350), (7, "['09-Jan-11', '16:15']", '일본', 7, 13, 68, 2, 17, 17, 4180995), (8, "['21-Jan-11', '16:25']", '일본', 6, 17, 59, 1, 17, 17, 4180995), (9, "['25-Jan-11', '16:25']", '일본', 8, 35, 57, 4, 17, 17, 4180995), (10, "['13-Jan-11', '19:15']", '일본', 5, 19, 59, 4, 19, 17, 4180995), (11, "['17-Jan-11', '16:15']", '일본', 9, 23, 52, 4, 20, 17, 4180995), (12, "['29-Jan-11', '18:00']", '일본', 3, 14, 55, 4, 20, 17, 4180995), (13, "['09-Jan-11', '19:15']", '사우디아라비아', 3, 13, 55, 1, 24, 98, 577598), (14, "['17-Jan-11', '16:15']", '사우디아라비아', 3, 7, 48, 4, 20, 98, 577598), (15, "['13-Jan-11', '16:15']", '사우디아라비

In [72]:
# 2011년 데이터 프레임화 시키기
Asian_M2011= pd.DataFrame(list(rows))
Asian_M2011=Asian_M2011[[1,2,3,4,5,6,7,8,9]] # Seq 칼럼 지우기

In [73]:
# mysql Connection
conn = pymysql.connect(host = '192.168.101.106',user = 'root',password = '1111',db = 'asiancup',charset = 'utf8')

#Connection으로 부터 cursor생성 - 테이터 베이스 빈줄에 커서가 깜빡 거리고 있는 것이다.
curs = conn.cursor()

# SQL 문장
sql = 'select * from nations2015'
curs.execute(sql) # 실행문 - 결과는 데이터가 몇개 있는지 알려준다.

# Data Fetch
rows = curs.fetchall()
print(rows)
conn.close()

((1, "['31-Jan-15', '20:00']", '한국', 3, 19, 47, 4, 24, 51, 2306476), (2, "['30-Jan-15', '20:00']", '이라크', 7, 20, 37, 1, 24, 89, 9586), (3, "['27-Jan-15', '20:00']", '오스트레일리아', 3, 15, 53, 3, 19, 100, 1534325), (4, "['26-Jan-15', '20:00']", '한국', 7, 7, 53, 3, 20, 51, 2306476), (5, "['23-Jan-15', '20:30']", '일본', 8, 17, 68, 1, 27, 53, 7021535), (6, "['23-Jan-15', '17:30']", '이란', 4, 9, 36, 1, 31, 45, 936990), (7, "['22-Jan-15', '20:30']", '중국', 5, 12, 34, 3, 25, 84, 8151), (8, "['22-Jan-15', '18:30']", '한국', 7, 6, 61, 4, 34, 51, 2306476), (9, "['20-Jan-15', '20:00']", '일본', 9, 21, 66, 1, 29, 53, 7021535), (10, "['20-Jan-15', '20:00']", '이라크', 8, 15, 64, 1, 24, 89, 9586), (11, "['19-Jan-15', '20:00']", '카타르', 5, 11, 59, 1, 22, 86, 137367), (12, "['19-Jan-15', '19:00']", '이란', 4, 19, 32, 3, 28, 45, 936990), (13, "['18-Jan-15', '20:00']", '우즈베키스탄', 5, 29, 36, 1, 16, 74, 979110), (14, "['18-Jan-15', '20:00']", '중국', 4, 9, 64, 1, 25, 84, 8151), (15, "['17-Jan-15', '20:00']", '오만', 3, 16, 41, 1

In [74]:
# 2015년 데이터 프레임화 시키기
Asian_M2015= pd.DataFrame(list(rows))
Asian_M2015=Asian_M2015[[1,2,3,4,5,6,7,8,9]]

### 데이터 정제

In [75]:
# 데이터 확인
Asian_M2011.head()

,1,2,3,4,5,6,7,8,9
0,"['16-Jan-11', '19:15']",카타르,6,23,43,4,24,92,11295
1,"['12-Jan-11', '19:15']",카타르,4,16,51,1,21,92,11295
2,"['21-Jan-11', '16:25']",카타르,7,18,41,1,17,92,11295
3,"['08-Jan-11', '16:15']",쿠웨이트,2,17,51,1,23,96,10350
4,"['12-Jan-11', '16:15']",쿠웨이트,3,14,48,1,21,96,10350


In [76]:
# 데이터 확인
Asian_M2015.head()

,1,2,3,4,5,6,7,8,9
0,"['31-Jan-15', '20:00']",한국,3,19,47,4,24,51,2306476
1,"['30-Jan-15', '20:00']",이라크,7,20,37,1,24,89,9586
2,"['27-Jan-15', '20:00']",오스트레일리아,3,15,53,3,19,100,1534325
3,"['26-Jan-15', '20:00']",한국,7,7,53,3,20,51,2306476
4,"['23-Jan-15', '20:30']",일본,8,17,68,1,27,53,7021535


In [77]:
# 컬럼 이름 변경
Asian_M2011.columns = ['기간','국가','유효슈팅','반칙','점유율','날씨','기온','피파랭킹','팀연봉']
Asian_M2011.head()

,기간,국가,유효슈팅,반칙,점유율,날씨,기온,피파랭킹,팀연봉
0,"['16-Jan-11', '19:15']",카타르,6,23,43,4,24,92,11295
1,"['12-Jan-11', '19:15']",카타르,4,16,51,1,21,92,11295
2,"['21-Jan-11', '16:25']",카타르,7,18,41,1,17,92,11295
3,"['08-Jan-11', '16:15']",쿠웨이트,2,17,51,1,23,96,10350
4,"['12-Jan-11', '16:15']",쿠웨이트,3,14,48,1,21,96,10350


In [78]:
# 컬럼 이름 변경
Asian_M2015.columns = ['기간','국가','유효슈팅','반칙','점유율','날씨','기온','피파랭킹','팀연봉']
Asian_M2015.head()

,기간,국가,유효슈팅,반칙,점유율,날씨,기온,피파랭킹,팀연봉
0,"['31-Jan-15', '20:00']",한국,3,19,47,4,24,51,2306476
1,"['30-Jan-15', '20:00']",이라크,7,20,37,1,24,89,9586
2,"['27-Jan-15', '20:00']",오스트레일리아,3,15,53,3,19,100,1534325
3,"['26-Jan-15', '20:00']",한국,7,7,53,3,20,51,2306476
4,"['23-Jan-15', '20:30']",일본,8,17,68,1,27,53,7021535


In [79]:
# 2011년 2015년 데이터 합치기
Asian_M2011_2015 = pd.concat([Asian_M2011,Asian_M2015])
Asian_M2011_2015.head()

,기간,국가,유효슈팅,반칙,점유율,날씨,기온,피파랭킹,팀연봉
0,"['16-Jan-11', '19:15']",카타르,6,23,43,4,24,92,11295
1,"['12-Jan-11', '19:15']",카타르,4,16,51,1,21,92,11295
2,"['21-Jan-11', '16:25']",카타르,7,18,41,1,17,92,11295
3,"['08-Jan-11', '16:15']",쿠웨이트,2,17,51,1,23,96,10350
4,"['12-Jan-11', '16:15']",쿠웨이트,3,14,48,1,21,96,10350


In [80]:
# 국가 카테고리 값 확인 - 한글 순서대로 카테고리 적용됨
natcat = pd.DataFrame(list(Asian_M2011_2015['국가'].unique()))
natcat = natcat.sort_values(by = 0)
# natcat.reindex(index = range(1,19), columns=[0])
natcat['순서'] = range(1,19)
natcat

,0,순서
13,바레인,1
6,북한,2
3,사우디아라비아,3
11,시리아,4
15,아랍에미리트,5
16,오만,6
12,오스트레일리아,7
10,요르단,8
8,우즈베키스탄,9
7,이라크,10


In [81]:
# Asian_M2011_2015['피파랭킹'] = np.log1p(Asian_M2011_2015['피파랭킹'])

In [82]:
# 국가를 카테고리화 시키기
Asian_M2011_2015['국가'] = Asian_M2011_2015['국가'].astype('category')
# 날씨데이터
Asian_M2011_2015['날씨'] = Asian_M2011_2015['날씨'].astype('category')

Asian_M2011_2015['국가'].cat.categories = ['%s'%i for i in range(1,19)]

In [83]:
Asian_M2011_2015.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 128 entries, 0 to 63
Data columns (total 9 columns):
기간      128 non-null object
국가      128 non-null category
유효슈팅    128 non-null int64
반칙      128 non-null int64
점유율     128 non-null int64
날씨      128 non-null category
기온      128 non-null int64
피파랭킹    128 non-null int64
팀연봉     128 non-null int64
dtypes: category(2), int64(6), object(1)
memory usage: 9.2+ KB


# 머신러닝

## 시도방법5

In [84]:
# 피파랭킹 값 변경 - 높을수록 좋은 값으로 - 아시안컵 최대값은 160
Asian_M2011_2015['피파랭킹'] = 211 - Asian_M2011_2015['피파랭킹']

In [85]:
# 반칙 횟수 최댓값
Asian_M2011_2015['반칙'].max()

35

In [86]:
# 반칙의 최대값은 35 이다. 40에서 빼서 사용해 보자
Asian_M2011_2015['반칙'] = 40 - Asian_M2011_2015['반칙']

In [87]:
# 팀 연봉의 최대값
Asian_M2011_2015['팀연봉'].max()

7021535

In [88]:
# 팀 연봉 데이터 차이 만들기
Smax = pd.DataFrame(list(Asian_M2011_2015['팀연봉'].unique())).sort_values(by=0).max() # 연봉의 최대값 7021535

# 최상위 선수들만 가산점을 주는 방식
Asian_M2011_2015['팀연봉'] / 7021535 * 40

0      0.064345
1      0.064345
2      0.064345
3      0.058961
4      0.058961
5      0.058961
6     23.818125
7     23.818125
8     23.818125
9     23.818125
10    23.818125
11    23.818125
12     3.290437
13     3.290437
14     3.290437
15     0.010351
16     0.010351
17     0.010351
18    13.153563
19    13.153563
20    13.153563
21    13.153563
22    13.153563
23    13.153563
24     1.328547
25     1.328547
26     1.328547
27     0.034853
28     0.034853
29     0.034853
        ...    
34     0.787184
35     0.054609
36     0.787184
37     0.054609
38     8.740681
39     5.577755
40     0.021659
41     0.010397
42     0.040384
43     0.787184
44     0.068373
45     0.024741
46     0.029327
47    13.139440
48    40.000000
49     0.021659
50     5.337807
51     0.787184
52     5.577755
53     0.068373
54     8.740681
55    13.139440
56     0.054609
57     0.010397
58     0.040384
59     0.782547
60     0.046434
61     0.024741
62     0.944848
63     0.029327
Name: 팀연봉, Length: 128, 

In [89]:
# 연봉데이터 변경
Asian_M2011_2015['팀연봉'] =  (Asian_M2011_2015['팀연봉'] / 6934420 * 40)

In [90]:
# 유효슈팅과 점유율에 가중치 주기 * 1.5
Asian_M2011_2015['유효슈팅'] = Asian_M2011_2015['유효슈팅'] * 1.5
Asian_M2011_2015['점유율'] = Asian_M2011_2015['점유율'] * 0.3

In [91]:
# 점수 최댓값 - 360.78839064392616
# score.max()

#### 팀별 점수내기

In [92]:
# 점수 내기!
score = Asian_M2011_2015['유효슈팅'] + Asian_M2011_2015['반칙'] + Asian_M2011_2015['점유율'] + Asian_M2011_2015['기온'] + \
+ Asian_M2011_2015['피파랭킹'] + Asian_M2011_2015['팀연봉']

# pd.DataFrame(list(score)).describe()
Asian_M2011_2015['점수'] = pd.DataFrame(list(score)) / 370 * 100
Asian_M2011_2015

,기간,국가,유효슈팅,반칙,점유율,날씨,기온,피파랭킹,팀연봉,점수
0,"['16-Jan-11', '19:15']",15,9.0,17,12.9,4,24,119,0.065153,49.179771
1,"['12-Jan-11', '19:15']",15,6.0,24,15.3,1,21,119,0.065153,50.098690
2,"['21-Jan-11', '16:25']",15,10.5,22,12.3,1,17,119,0.065153,48.882474
3,"['08-Jan-11', '16:15']",16,3.0,23,15.3,1,23,115,0.059702,48.475595
4,"['12-Jan-11', '16:15']",16,4.5,26,14.4,1,21,115,0.059702,48.908028
5,"['16-Jan-11', '19:15']",16,1.5,24,17.1,4,24,115,0.059702,49.097217
6,"['09-Jan-11', '16:15']",13,10.5,27,20.4,2,17,194,24.117345,79.193877
7,"['21-Jan-11', '16:25']",13,9.0,23,17.7,1,17,194,24.117345,76.977661
8,"['25-Jan-11', '16:25']",13,12.0,5,17.1,4,17,194,24.117345,72.761445
9,"['13-Jan-11', '19:15']",13,7.5,21,17.7,4,19,194,24.117345,76.572255


In [93]:
# 인덱스 재구성
Asian_M2011_2015.reset_index(inplace=True)

In [94]:
## 데이터 옮기기
train_A =  Asian_M2011_2015

# 필요한 Feature 구성
feature_name = \
['국가','날씨','기온','유효슈팅','반칙','점유율','피파랭킹','팀연봉']

In [95]:
# 훈련데이터 생성 - 문제
X_train = train_A[feature_name]
print(X_train.shape)
X_train.tail()

(128, 8)


,국가,날씨,기온,유효슈팅,반칙,점유율,피파랭킹,팀연봉
123,15,1,22,7.5,27,13.8,125,0.792378
124,14,3,28,3.0,26,13.2,127,0.047018
125,2,3,21,4.5,29,12.0,106,0.025052
126,6,3,19,1.5,32,9.9,107,0.956717
127,16,3,16,7.5,30,12.0,72,0.029695


In [96]:
# 훈련데이터 생성 - 답
Y_train = train_A['점수']
print(Y_train.shape)
Y_train.tail()

(128,)


123    64.183437
124    46.822316
125    42.579073
126    45.119614
127    49.612204
Name: 점수, dtype: float64

In [97]:
# RMSLE
from sklearn.metrics import make_scorer
def rmsle(predicted_values, actual_values):
    # Numpy로 배열 형태로 변경
    predicted_values = np.array(predicted_values)
    actual_values = np.array(actual_values)
    
    # 에측값과 실제값에 1을 더하고 log를 씌운다.
    log_predict = np.log(predicted_values + 1)
    log_actual = np.log(actual_values + 1)
    
    # 위에서 계싼된 예측값에서 실제값을 빼주고 제곱을 해준다.
    difference = log_predict - log_actual
    difference = np.square(difference)
    
    # 평균을 구한다
    mean_difference = difference.mean()
    
    # 루트를 씌운다
    score = np.sqrt(mean_difference)
    
    return score

rmsle_scorer = make_scorer(rmsle)
rmsle_scorer

make_scorer(rmsle)

In [98]:
k_fold = KFold(n_splits = 10, shuffle = True, random_state = 0) # 고정된 상태로 10번 섞어서 테스트해라

In [99]:
# 모델 생성
model = GradientBoostingRegressor(n_estimators=1000, alpha=0.1, random_state = 0) # 1000개부터 10프로씩 줄이면서 진행
model

GradientBoostingRegressor(alpha=0.1, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=1000, n_iter_no_change=None, presort='auto',
             random_state=0, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [100]:
# Training
model.fit(X_train, Y_train)

GradientBoostingRegressor(alpha=0.1, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=1000, n_iter_no_change=None, presort='auto',
             random_state=0, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [101]:
%time score = cross_val_score(model, X_train, Y_train, cv = k_fold, scoring = rmsle_scorer)
score = score.mean()
print("Score = ", score)

Wall time: 3.38 s
Score =  0.1874078245023402


## 팀점수예측 머신러닝

In [107]:
# 예측
# 날씨 - 구름 1 안개 2 비 3 맑음 4
print('바레인', '북한', '사우디아라비아', '시리아', '아랍에미리트', '오만', '오스트레일리아', '요르단')
print('우즈베키스탄', '이라크', '이란', '인도', '일본', '중국', '카타르', '쿠웨이트', '팔레스타인','한국')
print('')

# 랭킹 데이터 불러오기
Frank2019 = pd.read_csv('./data/FIFARank.csv')
Salary2019 = pd.read_csv('./data/players_2019_edit.csv')

print('')
print('오늘의 날씨를 입력해주세요!!','구름', '안개', '비', '맑음')

weather1 = input("날씨(구름', '안개', '비', '맑음) : ")
if weather1 == '구름':
    weather1 = 1
if weather1 == '안개':
    weather1 = 2
if weather1 == '비':
    weather1 = 3
if weather1 == '맑음':
    weather1 = 4
temp1 = input("오늘의 기온 : ")

print('')

# 팀 입력받기
team1 = input("팀 1 국가 : ")
a = natcat.loc[natcat[0]=='%s'%team1,"순서"]
a = int(a)
# a = Asian_M2011_2015.loc[Asian_M2011_2015['국가'] == '%d'%a,"국가"].unique()

rank1 = Frank2019.loc[Frank2019['국가'] == '%s'%team1,"랭킹"]
rank1 = int(rank1)

Salary1 = Salary2019.loc[Salary2019['국가'] == '%s'%team1,"팀연봉"]
Salary1 = int(Salary1)

print('팀 1 국가 %s의 랭킹 %d위가 점수에 반영되었습니다. '%(team1,rank1))
print('팀 1 국가 %s의 전체선수연봉 %d(1,000 단위)가 점수에 반영되었습니다. '%(team1,Salary1))

shoot1 = input("팀 1 유효슈팅 : ")
foul1 = input("팀 1 반칙 : ")
time1 = input("팀 1 점유율 : ")

print('')

# 팀2
team2 = input("팀 2 국가 : ")
b = natcat.loc[natcat[0]=='%s'%team2,"순서"]
b=int(b)
# b = Asian_M2011_2015.loc[Asian_M2011_2015['국가'] == '%d'%b,"국가"].unique()

rank2 = Frank2019.loc[Frank2019['국가'] == '%s'%team2,"랭킹"]
rank2 = int(rank2)

Salary2 = Salary2019.loc[Salary2019['국가'] == '%s'%team2,"팀연봉"]
Salary2 = int(Salary2)

print('팀 2 국가 %s의 랭킹 %d위가 점수에 반영되었습니다. '%(team2,rank2))
print('팀 2 국가 %s의 전체선수연봉 %d(1,000 단위)가 점수에 반영되었습니다. '%(team2,Salary2))

shoot2 = input("팀 2 유효슈팅 : ")
foul2 = input("팀 2 반칙 : ")
time2 = input("팀 2 점유율 : ")


pre1 = model.predict([[str(a),str(weather1),int(temp1),(int(shoot1) * 1.5),(40 - int(foul1)),(int(time1) * 0.3),(211 - rank1),(Salary1 / 6934420 * 40)]])
pre2 = model.predict([[str(b),str(weather1),int(temp1),(int(shoot2) * 1.5),(40 - int(foul2)),(int(time2) * 0.3),(211 - rank2),(Salary2 / 6934420 * 40)]])
print('')
print(team1, pre1) # 쿠웨이트
print(team2, pre2) # 우즈벡

if pre1 > pre2:
    print('%s (%d) 이 %s (%d)를 이겼습니다.'%(team1,pre1,team2,pre2))
if pre2 > pre1:
    print('%s (%d) 이 %s (%d)를 이겼습니다.'%(team2,pre2,team1,pre1))
    


바레인 북한 사우디아라비아 시리아 아랍에미리트 오만 오스트레일리아 요르단
우즈베키스탄 이라크 이란 인도 일본 중국 카타르 쿠웨이트 팔레스타인 한국


오늘의 날씨를 입력해주세요!! 구름 안개 비 맑음
날씨(구름', '안개', '비', '맑음) : 구름
오늘의 기온 : 24

팀 1 국가 : 중국
팀 1 국가 중국의 랭킹 76위가 점수에 반영되었습니다. 
팀 1 국가 중국의 전체선수연봉 326027(1,000 단위)가 점수에 반영되었습니다. 
팀 1 유효슈팅 : 1
팀 1 반칙 : 14
팀 1 점유율 : 48

팀 2 국가 : 이란
팀 2 국가 이란의 랭킹 29위가 점수에 반영되었습니다. 
팀 2 국가 이란의 전체선수연봉 4530110(1,000 단위)가 점수에 반영되었습니다. 
팀 2 유효슈팅 : 7
팀 2 반칙 : 18
팀 2 점유율 : 52

중국 [55.25455959]
이란 [72.27229136]
이란 (72) 이 중국 (55)를 이겼습니다.


> ## 시도방법 1

>> score = Asian_M2011_2015['유효슈팅'] + Asian_M2011_2015['반칙'] + Asian_M2011_2015['점유율'] + Asian_M2011_2015['기온'] + \
>> Asian_M2011_2015['기온'] + Asian_M2011_2015['피파랭킹'] + np.log1p(Asian_M2011_2015['팀연봉']) 
>> - 팀 연봉 데이터를 그대로 가지고 점수를 내면 팀 연봉의 영향력이 너무 크므로 표준화 시켜 점수에 적용
>> - 머신러닝에 들어가는 팀연봉 데이터는 표준화 시키지 않고, 모든 칼럼을 그대로 머신러닝에 사용

 Wall time: 5.72 s
Score =  0.22199101780680658

> ## 시도방법 2
>> - 점수게산에서 다른 방향으로 적용되는 점수들을 한방향으로 맞춰주지 않았다.
>> - (유효슈팅, 점유율, 팀연봉) - 높아질수록 승률 높아지는
>> - (반칙, 피파랭킹) - 낮을수록 승률이 높아지는 - > 낮을 수록 좋은 값을 반대로 맞춰주자!
>> - (기온) - 한경기에서 같은 값을 가질 수 밖에 없는 값 - 그냥 더해주자

Wall time: 2.58 s
Score =  0.1930022844021758

> ## 시도방법 3
>> - 시도2에서 연봉을 표준화 시켜서 머신러닝에 넣고 / 모든 데이터를 가지고 훈련시켜 임이의 값을 넣어 결과를 살펴보자 

Wall time: 5.05 s
Score =  0.1795432166027756

> ## 시도방법 4
>> - 시도3에서 연봉을 표준화 시켜서 점수에 반영하니 이 요소의 반영정도가 너무 낮아져 강팀의 강점이 부각되지 않는다.
>> - 표준화 시킨 값을 증폭시켜 어느정도 차이를 만들어낼 요소가 필요하다.

> ## 시도방법 5
>> - 현재의 방법으로는 연봉과 피파랭킹이 경기 승패에 커다란 요인이 되고 있다. 그래서 사우디 - 시리아 (2011.1.9)경기 맞추지 못함
>> - 각 경기의 우위인 볼점유율과 유효슈팅에 대한 점수 가중치가 필요하다.
>> - 점수에 가중치를 두어 계산 - 사용자입력에서도 가중치르 두어 입력
>> - ( 피파랭킹 값 변경 - 높을수록 좋은 값으로 - 아시안컵 최대값은 160 ) 
>>> Asian_M2011_2015['피파랭킹'] = 211 - Asian_M2011_2015['피파랭킹']
>> - ( 반칙의 최대값은 35 이다. 40에서 빼서 사용해 보자. )
>>> Asian_M2011_2015['반칙'] = 40 - Asian_M2011_2015['반칙']
>> - ( 최상위 선수들만 가산점을 주는 방식 )
>>> Asian_M2011_2015['팀연봉'] / 7021535 * 10
>> - ( 유효슈팅과 점유율에 가중치 주기 * 1.5 )
>>> Asian_M2011_2015['유효슈팅'] = Asian_M2011_2015['유효슈팅'] * 1.5   
>>> Asian_M2011_2015['점유율'] = Asian_M2011_2015['점유율'] * 1.5    